In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from webdriver_manager.chrome import ChromeDriverManager
import time
import mysql.connector


In [ ]:
# MySQL connection setup
db = mysql.connector.connect(
    host="127.0.0.1",  # Typically 'localhost' or '127.0.0.1'
    user="root",
    password="kjhyun0221",
    database="news"
)
cursor = db.cursor()

# Create table if it doesn't exist
cursor.execute("""
CREATE TABLE IF NOT EXISTS wti_books (
    id INT AUTO_INCREMENT PRIMARY KEY,
    title VARCHAR(255),
    url VARCHAR(255),
    published_date DATE
)
""")

def insert_data(title, url, published_date):
    # Remove the trailing period and any spaces
    published_date = published_date.strip().rstrip('.')
    
    # Split the date string by periods
    year, month, day = published_date.split('.')
    
    # Remove any leading or trailing spaces that may have been left over
    year = year.strip()
    month = month.strip()
    day = day.strip()
    
    # Construct the MySQL-compatible date string
    mysql_date = f"{year}-{month.zfill(2)}-{day.zfill(2)}"
    
    query = "INSERT INTO wti_books (title, url, published_date) VALUES (%s, %s, %s)"
    cursor.execute(query, (title, url, mysql_date))
    db.commit()

In [ ]:

chromedriver_path = 'C:/Users/kkksk/.wdm/drivers/chromedriver/win64/122.0.6261.129/chromedriver.exe'
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)

In [ ]:
target_year = 2022

current_year = 2024  

try:
    while target_year <= current_year:
        
        url = f"https://www.google.com/search?q=crude+oil&tbs=cdr:1,cd_min:1/1/{target_year},cd_max:12/31/{target_year}&tbm=nws"
        driver.get(url)
        
        while True:
            news_results = driver.find_elements(By.CSS_SELECTOR, 'div#rso > div >div>div>div')
            for news_div in news_results:
                try:
                    news_link = news_div.find_element(By.TAG_NAME, 'a').get_attribute('href')
                    news_text_elements = news_div.find_elements(By.CSS_SELECTOR, 'a>div>div>div')
                    news_item = [elem.text for elem in news_text_elements]
                    if len(news_item) >= 5:
                        Link = news_link
                        Title = news_item[2]
                        Date = news_item[4]
                        insert_data(Title,Link,Date)
                except Exception as e:
                    print(e)
            
            try:
                next_button = driver.find_element(By.ID, 'pnnext')
                next_button.click()
                time.sleep(2)
            except NoSuchElementException:
                break
        
        target_year += 1

except Exception as e:
    print(f"Encountered an exception: {e}")

finally:
    driver.quit()
